# A Demonstration of the RSA Cryptosystem

The Rivest&ndash;Shamir&ndash;Adleman ([RSA](https://en.wikipedia.org/wiki/RSA_(cryptosystem))) cryptosystem demonstrated here combines secure key&ndash;exchange with digital signature to provide for the _confidentiality_, _integrity_ and _authenticity_ of a private message exchanged between two parties (Alice and Bob) over an insecure channel. Moreover, an adversary (Eve) listening on this channel has no practical way of learning the content of the message, nor any way of tampering with it without the receiving party knowing the message has been corrupted.

The key&ndash;exchange component of this cryptosystem provides for message confidentiality utilizing a [_key&ndash;encapsulation mechanism_](https://en.wikipedia.org/wiki/Key_encapsulation_mechanism) (or _KEM_). This differs from more traditional approaches to key negotiation that utilize a key&ndash;_agreement_ mechanism, such as those based on the [discrete logarithm problem](https://en.wikipedia.org/wiki/Discrete_logarithm) introduced by Diffie and Hellman in their seminal 1976 paper [_New Directions in Cryptography_](https://ee.stanford.edu/~hellman/publications/24.pdf).

The digital signature component of this cryptosystem provides for message integrity and authenticity, ideas that were conceived by Diffie and Hellman in their 1976 paper, but not implemented until Rivest, Shamir and Adleman published [_A Method for Obtaining Digital Signatures and Public&ndash;Key Cryptosystems_](https://people.csail.mit.edu/rivest/Rsapaper.pdf) in 1978.

## The Source Code
The following sequence of function calls to the [rsa](https://github.com/dchampion/crypto/tree/master/src/core/rsa.py) module illustrates a session in which Alice signs and encrypts a private message, and then transmits the message to Bob over an insecure channel. Meanwhile, an adversary (Eve) intercepts the message.

It is assumed that Alice, Bob and Eve all possess their own copy of the [rsa](https://github.com/dchampion/crypto/tree/master/src/core/rsa.py) module; but that only Alice and Bob possess the signing and encryption keys produced by this module to provide the aforementioned protections.

Alice and Bob both import the `rsa` module, and a module `sym` that implements a symmetric cipher they have agreed in advance will be used to encrypt and decrypt messages (the `sym` cipher used here is for illustrative purposes only, and in a real&ndash;world setting would be replaced with a cryptographically&ndash;strong symmetric cipher such as _3DES_ or _AES_).

In [1]:
import os
os.chdir("../src")

# Alice and Bob (and possibly Eve) import the rsa and sym modules required for the session
from core import rsa
from tests.core import sym

Alice begins by generating her RSA keys. She does this by calling the `generate_rsa_key` function, passing it a modulus size of 2048 bits. The function returns the quintuple $pA$, $qA$, $nA$, $d3A$ and $d5A$ (the suffix $A$ on all the parameters identify them as belonging to Alice). Then Alice transmits her public modulus $nA$ to Bob.


In [2]:
# Alice computes her keys, and transmits the public key (nA) to Bob
pA, qA, nA, d3A, d5A = rsa.generate_rsa_key(2048)

Alice must keep the values $pA$ and $qA$&mdash;the 1024&ndash;bit prime factors of $nA$&mdash;secret. She must also keep $d3A$ and $d5A$&mdash;her signing and decryption keys, respectively&mdash;secret.

To get an idea of the size and proportions of these parameters, let's print some of them out.

In [3]:
print(pA)

128747458866473454469019166401278224362187678248677750686346562091632709204420827710005237104936737933412199698451977573748327818980245315853937746634188026821721642801361705117705723622775417422192537243819767928367430706527268099995133404475833535030517939290847438754879143167517341040139809848700297711023


In [4]:
print(qA)

134522086056917158097123968368048612202105825006186846903489921319520083764748764055353003919566906269945164692274768186699597076406105799034906033973255200351924949210632619938866320210129223905432185021482398577596284984785451601608787844794249716038041573543419637350967832861475801949683861458262378982799


In [5]:
print(nA)

17319376741245144029704515271514037431719084252261888863367140926529684779390987888625521455296371950721787106505539989635637378043616055272339140531613394989790623851945518086568717538409368027560958381031793280888036950815711952611324013001370981081588296795296005522947786750461270749207185645108212785871062949797068720897924371290680729895308053416668315139866520516846236973434834229645743241613338128788407691259215583929997876251317211213098296445132156156169445389378863677159652000886246514203711944464982735154860036420578561521064862161796997122566064355742340299229275781867987118368517657160682789693377


Note that $nA$, which is the product of $pA$ and $qA$, is twice the length of each individually; $pA$'s and $qA$'s ~310 digits to $nA$'s ~620.

Recall that Alice requested a modulus of 2048 bits in length in her call to `generate_rsa_key`. Next, let's print the bit lengths of $nA$ and its prime factors.

In [6]:
print(nA.bit_length())

2048


In [7]:
print(pA.bit_length())

1024


In [8]:
print(qA.bit_length())

1024


What we see is that a decimal number with a bit&ndash;length of 2048 consists of ~620 digits, and that a decimal number with a bit length of 1024 consists of ~310 digits.

Next, we prove that $nA$ is in fact the product of $pA$ and $qA$.

In [9]:
print(qA * pA == nA)

True


Now it's Bob's turn to generate RSA keys, and transmit his public key $nB$ to Alice. To differentiate Bob's keys from Alice's, we append Bob's with the captial letter $B$.

In [10]:
# Bob computes his keys, and transmits the public key (nB) to Alice
pB, qB, nB, d3B, d5B = rsa.generate_rsa_key(2048)

Now that Alice and Bob have generated their respective keys, and transmitted to each other their public moduli $nA$ and $nB$, they can proceed with the secure message exchange.

Let's say Alice wants to send the following message securely to Bob. First, Alice stores the message "Sign and encrypt me!" in the variable $mA$.

In [11]:
# Alice's message to Bob
mA = "Sign and encrypt me!"

Then Alice calls the `sign` fuction, passing it her secret signing key $d3A$, the two factors of $nA$, $pA$ and $qA$, and the message $mA$, to sign the message.
She stores the signature in the variable $oA$.

In [12]:
# Alice signs the message, and transmits the signature (oA) to Bob
oA = rsa.sign(d3A, pA, qA, mA)

Next, Alice calls the `encrypt_random_key` function, passing it Bob's RSA modulus $nB$ and the global encryption exponent _rsa.ENCRYPTION\_EXPONENT_. This computes a session key that will be used to encrypt the private message using a symmetric cipher (e.g., _3DES_ or _AES_). The function returns the pair $KA$ and $cA$, which are, respectively, the symmetric encryption (or _session_) key, and the ciphertext of its input material; the same material Bob will use to replicate the session key on his side of the channel.

In [13]:
# Alice computes a session key, and transmits its ciphertext (cA) to Bob
KA, cA = rsa.encrypt_random_key(nB, rsa.ENCRYPTION_EXPONENT)

Alice must keep the session key $KA$ private, but she transmits its ciphertext $cA$ to Bob.

Next, Alice encrypts the private message, calling the `encrypt` function of the symmetric cipher, and passing it her symmetric key $KA$ and the plaintext of her message $mA$.

The result of this operation is the ciphertext $mAC$.

In [14]:
# Alice encrypts the message, and transmits its ciphertext (mAC) to Bob
mAC = sym.encrypt(KA, mA)

Here we print the ciphertext to demonstrate that, if Eve were to intercept it, she would see nothing but a seemingly random string of numbers.

In [15]:
print(mAC)

27893658004493771326973239368190066486150309808394860216549615758132348555127


Next, Alice transmits the message signature $oA$, the message ciphertext $mAC$ and the ciphertext of the symmetric session key $cA$ to Bob.

In order to decrypt the message, Bob must first recover the session key Alice used to encrypt it. He does this by calling  `decrypt_random_key`, passing it his private decryption key $d5B$, the ciphertext of the symmetric key that Alice sent to him $cA$, and the two prime factors of his public modulus $pB$ and $qB$.

In [16]:
# Bob recovers the session key from its ciphertext
KB = rsa.decrypt_random_key(d5B, cA, pB, qB)

In order to decrypt the message, Bob must first recover the session key Alice used to encrypt it. He does this by calling  `decrypt_random_key`, passing it his private decryption key $d5B$, the ciphertext of the symmetric key that Alice sent to him $cA$, and the two prime factors of his public modulus $pB$ and $qB$.

Here we prove that the key Bob decrypted above, $KB$, is the same as the key Alice computed, $KA$.

In [17]:
print(KB == KA)

True


Bob calls `decrypt`, passing it $KB$ and the message ciphertext $mAC$ Alice sent to him, to recover Alice's plaintext message $mA$. He stores the result in $mB$.

In [18]:
# Bob decrypts the message cipertext
mB = sym.decrypt(KB, mAC)

Here we prove that the recovered plaintext indeed matches Alice's original message.

In [19]:
print(mB == mA)

True


If we print $mB$, we see that it matches Alice's message $mA$.

In [20]:
print(mB)

Sign and encrypt me!


In order to complete the session, all that remains is for Bob to verify Alice's signature, and thereby prove that the message has not been altered and, further, that it originated from Alice (or, at least, that it was signed with Alice's private key).

Bob calls the `verify` fuction, passing Alice's public modulus $nA$, the constant signature&ndash;verification key _rsa.VERIFICATION\_EXPONENT_, the decrypted message $mB$ and the sigature sent to him by Alice $oA$.


In [21]:
# Bob verifies the message signature
print(rsa.verify(nA, rsa.VERIFICATION_EXPONENT, mB, oA))

True


If the assertion above holds, Bob can be assured that the message "Sign and encrypt me!", even if intercepted by Eve, is known only to him and Alice, and that the message was not corrupted in any way by Eve.